Auto-Clean-Test

In [1]:
import logging
import os
import shutil
import func_timeout 
import dill as pickle
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from streamline.runners.auto_runner import AutoRunner
from streamline.runners.clean_optimize_runner import OptimizeClean

/Users/gabrielketron/AutoTest/AutoTestEnv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open("./data/DigenData/all_new_test_sets.pkl",'rb') as f:
    digennoise = pickle.load(f)

#print(digennoise.keys())

with open("./data/DigenData/all_new_test_sets_no_noise.pkl",'rb') as f:
    digen = pickle.load(f)

#print(digen.keys())
#print(digen['digen8_4426'])
#Test= pd.DataFrame(digennoise['digen8_4426']['X'])
#yval = pd.DataFrame(digennoise['digen8_4426']['y'])
#current_noise = digennoise['digen8_4426']['noise']
#yval.columns = ['y']
#yval.rename(columns = {'0':'y'}, inplace = True)
#out = pd.concat([Test, yval], axis=1)
#out.index.name = 'InstanceId'
#out.to_csv('./data/DigenStudy/digen8_4426.csv')
#print(out)

In [3]:

data = pd.DataFrame(digennoise['digen8_4426']['X'])
yval = pd.DataFrame(digennoise['digen8_4426']['y'])
current_noise = digennoise['digen8_4426']['noise']
yval.columns = ['y']
yval.rename(columns = {'0':'y'}, inplace = True)
out = pd.concat([data, yval], axis=1)
out.index.name = 'InstanceId'
digennoise['digen8_4426'] = current_noise
if current_noise == 0.0: 
    outstring = './data/DigenStudy/No_Noise/' + 'digen8_4426' + '.csv'
    #tempstring = './data/testfolder/' + 'digen8_4426' + '.csv'
    out.to_csv(outstring)
    #out.to_csv(tempstring)
else:
    outstring = './data/DigenStudy/Noisy/' + 'digen8_4426' + '.csv'
    #tempstring = './data/testfolder/' + 'digen8_4426' + '.csv'
    out.to_csv(outstring)
    #out.to_csv(tempstring)

X_train, X_test,y_train, y_test = train_test_split(data, yval,test_size=0.08, shuffle=True)

train_set = pd.concat([X_train, y_train], axis =1)
train_set.index.name = 'InstanceId'
tempstring = './data/testfolder/' + 'digen8_4426' +'_train.csv'
train_set.to_csv(tempstring)
cleanrun = OptimizeClean(dataset_name=['digen8_4426_train'], n_trials=5, data_path='./data/testfolder', class_label='y', instance_label="InstanceId")
cleanrun.run(run_para=False)
optimal = cleanrun.best_param
current_auc = cleanrun.best_goal
os.remove(tempstring)


[I 2023-07-06 18:21:15,559] A new study created in memory with name: no-name-501ddbfe-4c27-4c1d-b1c7-d97bf49b8c4b
  0%|          | 0/5 [00:00<?, ?it/s]INFO: ------------------------------------------------------- 
INFO: Loading Dataset: digen8_4426_train
INFO: Validating and Identifying Feature Types...
INFO: Running Initial EDA:
INFO: Initial Data Counts: ----------------
INFO: Instance Count = 9200
INFO: Feature Count = 10
INFO:     Categorical  = 0
INFO:     Quantitative = 10
INFO: Missing Count = 0
INFO:     Missing Percent = 0.0
INFO: Class Counts: ----------------
INFO: Class Count Information
INFO: 
   Class  Instances
0      1       4763
1      0       4437
INFO: No textual categorical features, skipping label encoding
INFO: Running Feature Engineering
INFO: No Features with high missingness found
INFO: Not removing any features due to high missingness
INFO: No non-binary categorical features, skipping categorical encoding
INFO: Top 10 Correlated Features
INFO: 
   Removed_Feat

categorical cut = 5
sig cutoff = 0.0198
feature missing = 0.7
cleaning miss = 0.6
correlation removal = 0.9
partition method: random
n_splits = 9

424 min runtime for 1 dataset with 100 trials. 
= 7 hours.

In [1]:
print(cleanrun.best_model)


NameError: name 'cleanrun' is not defined

In [6]:
tempstring = './data/testfolder/' + 'digen8_4426' +'_test.csv'
test_set = pd.concat([X_test, y_test], axis =1)
test_set.index.name = 'InstanceId'
test_set.to_csv(tempstring)

most_recent_run = AutoRunner(**optimal, dataset_names=['digen8_4426_test'],data_path='./data/testfolder',
                                    gen_report=False, clean=False, class_label='y', 
                                    instance_label='InstanceId', ml_algorithms=["NB", "LR", "DT", "EN", "XGB", "LGB", "CGB", "SVM","GB", "RF"], 
                                    timeout=15,
                                    exclude=["ANN","KNN","GP", 'eLCS', 'XCS', "ExSTraCS"]) # "XGB", "LGB", "CGB", "SVM","GB", "RF"
output_csv = most_recent_run.run(run_para=True)
performance = pd.read_csv(output_csv)
summary_chart = performance
goal = performance["ROC AUC"].max()
best_model = performance.loc[performance["ROC AUC"].idxmax()][0]
png_out = output_csv.removesuffix('Summary_performance_mean.csv')
png_out = png_out + 'Summary_ROC.png'
final_model_comparison = plt.show(png_out)
print(current_auc-goal)
os.remove(tempstring)

INFO: ------------------------------------------------------- 
INFO: Loading Dataset: digen8_4426_test
  0%|          | 0/40 [00:00<?, ?it/s]ERROR:root:KeyError while copying model
ERROR:root:KeyError while copying model
100%|██████████| 40/40 [00:01<00:00, 36.00it/s]
ERROR:root:KeyError while copying model
ERROR:root:KeyError while copying model


0.11543951866760194


In [ ]:
summary_chart


,Unnamed: 0,Balanced Accuracy,Accuracy,F1 Score,Sensitivity (Recall),Specificity,Precision (PPV),TP,TN,FP,FN,NPV,LR+,LR-,ROC AUC,PRC AUC,PRC APS
0,Naive Bayes,0.475105,0.477514,0.390795,0.348668,0.601541,0.450009,15.000000,27.444444,18.333333,28.111111,0.495638,0.884116,1.087029,0.475588,0.481307,0.492785
1,Logistic Regression,0.444390,0.447466,0.405827,0.399115,0.489665,0.418214,17.333333,22.444444,23.333333,25.777778,0.465940,0.773164,1.225786,0.438939,0.442278,0.453926
2,Decision Tree,0.553911,0.557499,0.535308,0.535701,0.572122,0.541737,23.222222,26.333333,19.444444,19.888889,0.571132,1.272208,0.807631,0.570321,0.570490,0.543371
3,Elastic Net,0.455931,0.456234,0.403304,0.386964,0.524897,0.431356,16.666667,23.888889,21.888889,26.444444,0.477443,0.841518,1.191477,0.443323,0.460922,0.471854
4,Extreme Gradient Boosting,0.769463,0.771153,0.755283,0.742244,0.796683,0.772545,32.111111,36.444444,9.333333,11.000000,0.772799,4.334921,0.329382,0.834743,0.792646,0.799017
5,Light Gradient Boosting,0.944019,0.943806,0.942072,0.950518,0.937520,0.935384,41.000000,42.888889,2.888889,2.111111,0.954182,13.648610,0.054835,0.978071,0.971848,0.972339
6,Category Gradient Boosting,0.973538,0.973754,0.972291,0.961174,0.985901,0.983981,41.444444,45.111111,0.666667,1.666667,0.964502,8.964529,0.039661,0.998142,0.998091,0.998112
7,Support Vector Machine,0.771880,0.772458,0.759695,0.753458,0.790303,0.770335,32.555556,36.111111,9.666667,10.555556,0.776464,5.025416,0.343074,0.851916,0.857377,0.860230
8,Gradient Boosting,0.947416,0.947537,0.945692,0.952857,0.941975,0.942146,41.111111,43.111111,2.666667,2.000000,0.958933,9.841458,0.049436,0.984399,0.981672,0.981931
9,Random Forest,0.640936,0.642251,0.610066,0.585783,0.696088,0.641222,25.333333,31.777778,14.000000,17.777778,0.643753,2.138345,0.617937,0.689548,0.659714,0.668378


trials: 1 difference: 0.34 param: {'categorical_cutoff': 10, 'sig_cutoff': 0.02896993117424986, 'featureeng_missingness': 0.3, 'cleaning_missingness': 0.8, 'correlation_removal_threshold': 0.9, 'exploration_list': ['Describe', 'Univariate Analysis', 'Feature Correlation'], 'partition_method': 'Random', 'n_splits': 9}

trials: 10 difference: 0.35 param: {'categorical_cutoff': 9, 'sig_cutoff': 0.007051812409598334, 'featureeng_missingness': 0.8, 'cleaning_missingness': 0.15000000000000002, 'correlation_removal_threshold': 0.95, 'exploration_list': ['Describe', 'Univariate Analysis', 'Feature Correlation'], 'partition_method': 'Random', 'n_splits': 4}

In [ ]:
# autotest = AutoRunner(dataset_names=['digen8_4426'], clean=False, class_label='y', instance_label='InstanceId')

In [ ]:
#output_csv = autotest.run()

In [ ]:
#print(output_csv)

#performance = pd.read_csv(output_csv)
#print(performance)
#print(performance['ROC AUC'].max())
#print(performance.loc[performance['ROC AUC'].idxmax()][0])